# Описание мини-проекта: анализ бронирований отелей.

Подготовить датасет.

Найти 5 топ стран по количеству успешных бронирований.

Посчитать среднее значение бронирований в разных отелях.

Посчитать количество ситуаций в которых тип заказанного номера отличался от забронированного.

Сравнить по годам наиболее популярные для бронирования месяцы.

Найти месяц с максимальным количествтвом отмен брони в 2015, 2016 и 2017 годах с группировкой по отелям. 

Сравнить среднее значение числа взрослых, детей и младенцев.

Найти наиболее популярный отель среди гостей с наибольшим числом детей и младенцев.

Сравнить churn rate у гостей с детьми и без детей.

In [2]:
# Подготовка датасета:

import pandas as pd

df = pd.read_csv('а:\\hotels_bookings.csv', sep=';')

df.columns = [column.lower().replace(' ','_') for column in df.columns]

df.head(3)

,hotel,is_canceled,lead_time,arrival_full_date,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,adults,children,babies,meal,country,reserved_room_type,assigned_room_type,customer_type,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02


In [24]:
Нахождение 5 топ стран по количеству успешных бронирований:

        .groupby('country', as_index=False) \
        .agg({'hotel': 'count'}) \
        .sort_values('hotel', ascending=False) \
        .head(5)

,country,hotel
125,PRT,21071
57,GBR,9676
54,FRA,8481
50,ESP,6391
42,DEU,6069


In [3]:
# Расчет среднеего значения бронирований в разных отелях:

df.query('is_canceled == 0') \

df['total_nights'] = df['stays_in_weekend_nights'] + df['stays_in_week_nights']

df.groupby('hotel', as_index=False) \
    .agg({'total_nights': 'mean'}) \
    .round(2)

,hotel,total_nights
0,City Hotel,2.98
1,Resort Hotel,4.32


In [4]:
# Расчет количества ситуаций в которых тип заказанного номера отличался от забронированного:

Сравнить churn rate у гостей с детьми и без детей.

df.query('assigned_room_type != reserved_room_type') \
    .count() \
    .head(1)

hotel    14917
dtype: int64

In [31]:
Сравнение по годам наиболее популярных для бронирования месяцов.

df.query('arrival_date_year == 2016') \
        .groupby('arrival_date_month', as_index=False) \
        .agg({'is_canceled': 'count'}) \
        .rename(columns={'is_canceled': 'stays_num'}) \
        .sort_values('stays_num', ascending=False) \
        .head(1)

,arrival_date_month,stays_num
10,October,6203


In [11]:
df.query('arrival_date_year == 2017') \
        .groupby('arrival_date_month', as_index=False) \
        .agg({'is_canceled': 'count'}) \
        .rename(columns={'is_canceled': 'stays_num'}) \
        .sort_values('stays_num', ascending=False) \
        .head(1)

,arrival_date_month,stays_num
7,May,6313


In [26]:
# Нахождение месяца с максимальным количествтвом отмен брони в 2015, 2016 и 2017 годах с группировкой 
# по отелям. 

df.query('hotel == "City Hotel" and is_canceled == 1') \
        .groupby('arrival_date_year') \
        .agg({'arrival_date_month': 'value_counts'}) \
        .rename(columns={'arrival_date_month': 'cancellations_num'})

cancellations_num
arrival_date_year arrival_date_month                   
2015              September                        1543
                  October                          1321
                  August                           1232
                  July                              939
                  December                          668
                  November                          301
2016              October                          1947
                  June                             1720
                  September                        1567
                  April                            1539
                  May                              1436
                  November                         1360
                  August                           1247
                  March                            1108
                  December                         1072
                  July                             1043
                  February                          930
                  January                           438
2017              May                              2217
                  April                            1926
                  June                             1808
                  July                             1324
                  March                            1278
                  August                           1123
                  January                          1044
                  February                          971

In [35]:
# Сравнение средних значений числа взрослых, детей и младенцев:

df.describe().loc['mean'][['adults','children','babies']]

adults      1.856403
children    0.103890
babies      0.007949
Name: mean, dtype: float64

In [23]:
# Нахождение наиболее популярного отеля среди гостей с наибольшим числом детей и младенцев:

df['total_kids'] = df.children + df.babies

df.groupby('hotel', as_index=False) \
    .agg({'total_kids': 'mean'}) \
    .round(2)

,hotel,total_kids
0,City Hotel,0.10
1,Resort Hotel,0.14


In [52]:
# Сравнение показателя оттока для гостей с детьми и без детей:

df['has_kids'] = df.total_kids > 0

no_kids_rate = df.query('is_canceled == 1 and has_kids == False') \
                    .shape[0]/df.query('has_kids == False') \
                    .shape[0]

churn_rate = round(no_kids_rate * 100, 2)

churn_rate

37.22

In [55]:
has_kids_rate = df.query('is_canceled == 1 and has_kids == True') \
                    .shape[0]/df.query('has_kids == True') \
                    .shape[0]

churn_rate = round(has_kids_rate * 100, 2)
churn_rate

34.92